In [ ]:
import sys
sys.path.append('F:\Coding\SOIKA')
import torch
import pandas as pd
from factfinder import EventDetection, Geocoder, TextClassifier, TextClassifierTopics
import geopandas as gpd
import numpy as np
np.random.seed(42)

In [2]:
df_texts = pd.read_excel('F:\Coding\Сообщения на неделю.xlsx')

In [3]:
df_texts = df_texts[['Дата создания', 'Текст']].reset_index()
df_texts.columns = ['message_id', 'Дата и время', 'Текст комментария']
df_texts.head(5)

,message_id,Дата и время,Текст комментария
0,0,2022-12-01,"Здравствуйте.<br>У нас, жителей Санкт-Петербур..."
1,1,2022-12-01,И еще вопрос - мой муж инвалид 3 категории с 2...
2,2,2022-12-01,Что можно сказать обращайте на всякую ерунду п...
3,3,2022-12-01,Многие не видят но под этот шумок то что проис...
4,4,2022-12-01,Для какой цели произведена солевая обработка п...


In [4]:
device_type = torch.device('cuda:0')
classification_model = TextClassifier(
    repository_id="Sandrro/text_to_function_v2",
    number_of_categories=1,
    device_type=device_type,
)
df_categorized = df_texts.copy()
df_categorized[['cats','probs']] = pd.DataFrame(df_categorized['Текст комментария'].progress_map(lambda x: classification_model.run(x)).to_list())
df_categorized.dropna(subset='cats', inplace=True)
df_categorized.head(3)

 97%|█████████▋| 723/748 [00:05<00:00, 191.05it/s]

text is not string


100%|██████████| 748/748 [00:05<00:00, 127.58it/s]


,message_id,Дата и время,Текст комментария,cats,probs
0,0,2022-12-01,"Здравствуйте.<br>У нас, жителей Санкт-Петербур...",ЖКХ,0.999
1,1,2022-12-01,И еще вопрос - мой муж инвалид 3 категории с 2...,Здравоохранение,0.959
2,2,2022-12-01,Что можно сказать обращайте на всякую ерунду п...,Другое,0.994


In [5]:
df_geocoded = Geocoder().run(df_categorized, text_column='Текст комментария')
df_geocoded.head(3)

2023-09-04 20:00:03,533 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


100%|██████████| 341/341 [02:33<00:00,  2.21it/s]


,index,message_id,Дата и время,Текст комментария,cats,probs,Street,Numbers,Score,location_options,Location,geometry,level
0,0,0,2022-12-01,"Здравствуйте.<br>У нас, жителей Санкт-Петербур...",ЖКХ,0.999,ветеранов,152,0.99,"['мост Ветеранов 152 Санкт-Петербург Россия', ...","мост Ветеранов, Лигово, округ Урицк, Санкт-Пет...",POINT (30.18708 59.83306),house
1,0,0,2022-12-01,"Здравствуйте.<br>У нас, жителей Санкт-Петербур...",ЖКХ,0.999,ветеранов,152,0.99,"['мост Ветеранов 152 Санкт-Петербург Россия', ...","152 к5, проспект Ветеранов, округ Константинов...",POINT (30.15092 59.83357),house
2,1,1,2022-12-01,И еще вопрос - мой муж инвалид 3 категории с 2...,Здравоохранение,0.959,NaN,NaN,NaN,NaN,NaN,POINT (30.32964 59.92113),global


In [6]:
df_geocoded.drop(columns=['Дата и время']).explore()

In [ ]:
event_model = EventDetection()
messages, events, connections = event_model.run(df_geocoded, 'F:/Coding/SOIKA/data/raw/population.geojson', 'Санкт-Петербург', 32636, min_event_size=3)

In [8]:
events

,name,docs,level,id,risk,message_ids,geometry
0,0_не_на_quot_по,"481, 335, 445",link,0_link_19653.0,0.042308,"481, 0, 335, 202, 98, 445",POINT (30.19918 59.83347)
1,1_ветеранов_не_жильцы_по,"428, 283, 494",link,1_link_19653.0,0.025333,"428, 283, 494",POINT (30.18708 59.83306)
0,0_не_на_quot_по,"481, 335, 445",link,0_link_19693.0,0.042308,"481, 0, 335, 202, 98, 445",POINT (30.19918 59.83347)
1,1_ветеранов_не_жильцы_по,"428, 283, 494",link,1_link_19693.0,0.025333,"428, 283, 494",POINT (30.18708 59.83306)
0,0_лифт_топить_постоянно_будет,"497, 349, 178",link,0_link_12120.0,0.119822,"497, 692, 575, 190, 188, 226, 230, 377, 270, 7...",POINT (30.32964 59.92113)
...,...,...,...,...,...,...,...
160,160_дает спать по_ветеранов 47_заставить демон...,"283, 737, 484",global,160_global_0,0.019385,"283, 737, 484",POINT (30.32964 59.92113)
161,161_лестничной клетки_клетки_лестничной_лифт,"353, 298, 694",global,161_global_0,0.026667,"353, 298, 694",POINT (30.32964 59.92113)
162,162_наверно_не убирал_убирал_на солидарности 10к1,"315, 306, 682",global,162_global_0,0.008462,"315, 306, 561, 682",POINT (30.32964 59.92113)
163,163_строительство_наш_убрали_коммуникации,"332, 427, 451",global,163_global_0,0.022944,"332, 427, 451",POINT (30.32964 59.92113)


In [9]:
events.explore()

In [10]:
connections

,weight,a,b,geometry
0,12,0_global_0,1_global_0,"LINESTRING (30.321 59.862, 30.263 59.892)"
1,11,0_global_0,2_global_0,"LINESTRING (30.321 59.862, 30.362 59.931)"
2,10,0_global_0,3_global_0,"LINESTRING (30.321 59.862, 30.330 59.921)"
3,12,0_global_0,4_global_0,"LINESTRING (30.321 59.862, 30.272 59.840)"
4,11,0_global_0,5_global_0,"LINESTRING (30.321 59.862, 30.330 59.921)"
...,...,...,...,...
67523,11,0_link_17075.0,0_link_13266.0,"LINESTRING (30.292 59.848, 30.292 59.848)"
67524,6,0_link_17075.0,1_link_13266.0,"LINESTRING (30.292 59.848, 30.292 59.848)"
67525,6,1_link_17075.0,0_link_13266.0,"LINESTRING (30.292 59.848, 30.292 59.848)"
67526,7,1_link_17075.0,1_link_13266.0,"LINESTRING (30.292 59.848, 30.292 59.848)"
